# Demand side flexibility


In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

    Updating registry at `C:\Users\zoele\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [3]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [4]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [5]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [6]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame) ./ SCALE_MWH;

In [7]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [8]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [9]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 4)


Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [10]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [11]:
curtailment_costs = [0, 2, 2, 2, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.002
 0.002
 0.0
 0.0

Test without storage first

In [158]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 4
n_eu = 4
# pad year by 1 day
add_h = 24

C_FC = fixed_cost[1:techs,1]
println(C_FC)
C_FOM = fixed_cost[1:techs,2]
println(C_FOM)
E_CO2 = co2_intensity[1:techs,1]
C_VOM = variable_cost[1:techs,1]
println(C_VOM)
C_ramp = variable_cost[1:techs,2]
C_start = variable_cost[1:techs,3]
C_curt = curtailment_costs[1:techs]

D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
println(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_r = ramp_limits[1:techs, 1]

A_s = Availability_matrix[sel_hrs,1:techs]
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:n_eu]), A_s, A_s[1:24, 1:n_eu])
A_s[:, 1] = A_s[:, 1] * 0.25
A_s[:, 2] = A_s[:, 2] * 0.1
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
[543.5, 666.8212034, 1110.5, 3415.0]
[21.75309238, 28.779521369999998, 37.5, 103.5]
[0.0, 0.0, 0.005889999999999999, 0.011176]
(1001, 4)
(1049, 4)
(1049, 4)


# Constants

In [159]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton

# Model

In [160]:
model = Model(Gurobi.Optimizer) # TODO: options? other optimizers?
set_optimizer_attribute(model, "NonConvex", 2)
# set_optimizer_attribute(model, "IterationLimit", 10000)
println("Model: initialized!\n")

n = size(C_FC, 1) # number of technologies
println("Number of technologies: ", n)
T = size(D_b, 1) # hours in simulation
println("Timespan: ", T)
time_ls = 1+add_h:T-add_h
vre_ls = 1:2
fossil_ls = 3:4

# Non-negativity constraints
# The installed power capacity of the mix
@variable(model, P[1:n] >= 0)

# The hourly power generation of each technology
@variable(model, X_gen[1:T, 1:n] >= 0); # hourly production/use for each technology

# Curtailment (only renewables?)
@variable(model, X_cur[1:T, 1:n] >= 0); # hourly production/use for each technology

###### CONSTRAINTS ######

# Demand constraint - meet demand per hour
println("Demand constraint")
# @constraint(model, Con_dh[t = 1:T], sum(X_gen[t, i] for i in 1:n) >= X_D[t]); # TODO: add curtailment
# Meet total demand
@constraint(model, Con_d, sum(X_gen[t, i] for i in 1:n for t in 1:T) >= sum(total_demand));

# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s);
@constraint(model, Con_wgen, P[2] <= P_max_w);

# Cannot produce more than the installed power
# println("Installed power constraint")
# @constraint(model, Con_p[i = 1:n, t = 1:T], X_gen[t, i] <= P[i])

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = 1:T], X_gen[t, i] <= P[i]*A_s[t, i])
# Renewables produce maximum possible capacity
# @constraint(model, [i = fossil_ls, t = 1:T], X_gen[t, i] <= P[i]) 
# @constraint(model, [i = vre_ls, t = 1:T], X_gen[t, i] == P[i]*A_s[t, i]) 

println("Curtailment definition")
@constraint(model, [t = 1:T], sum(X_cur[t, i] for i in 1:n) == sum(X_gen[t, i] for i in 1:n) - X_D[t]) 
@constraint(model, [i = 1:n, t = 1:T], X_cur[t, i] <= X_gen[t, i])

# Cannot produce less than the minimum stable generation: X_gen must be zero or min to max
# min_usage = [0.1, 0.1, 0.1, 0.1]
# @constraint(model, [i = 1:n, t = 1:T], X_gen[t, i] >= P[i]*A_s[t, i]*min_usage[i])
# HACK
# @constraint(model, [i = vre_ls], P[i] >= 100)

# For each tech, at each hour, change in production cannot exceed ramp rate (TODO might need to convert this to time)
println("Ramp constraint")
# Dummy rate of change variable
# @variable(model, X_roc[time_ls, 1:n])
# @constraint(model, [i = 1:4, t = time_ls], X_roc[t, i] == X_gen[t, i]-X_gen[limit_bound_max(t+1, T), i])

# Auxiliary variables to linearize the rate of change constraint
@constraint(model, Con_rdwn[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) >= -A_r[i]*X_gen[t, i]) 
@constraint(model, Con_rup[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) <= A_r[i]*X_gen[t+1, i]);

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-12-11
Set parameter NonConvex to value 2
Model: initialized!

Number of technologies: 4
Timespan: 1049
Demand constraint


Renewable maximum installed capacity constraint
Availability & installed power constraint
Curtailment definition
Ramp constraint


## Objective

In [161]:
# # Add objective function: minimize the total costs TODO: discount rates
# function start_cost(x_0, x_1, i)
#     if x_0 == 0
#         C_start[i]
#     else
#         0
#     end
# end

#TODO RAMP AND STARTUP COSTS

@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
        sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls) +
        sum(sum(C_curt[i] * X_cur[t, i] for i in 1:n) for t in time_ls)
));

In [162]:
#Optimize
optimize!(model)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17452 rows, 8396 columns and 43190 nonzeros
Model fingerprint: 0xce17dfa9
Coefficient statistics:
  Matrix range     [4e-10, 1e+00]
  Objective range  [2e-03, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 8e+05]
Presolve removed 3267 rows and 1263 columns
Presolve time: 0.08s
Presolved: 14185 rows, 7133 columns, 41700 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 4
 AA' NZ     : 8.723e+04
 Factor NZ  : 2.105e+05 (roughly 10 MB of memory)
 Factor Ops : 3.266e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Pri

## Results

In [163]:
value.(P)

4-element Vector{Float64}:
  2050.3944493904705
 11158.008
   820.3927263558139
     0.0

In [164]:
X_gen_matrix = value.(X_gen);
    X_gen_matrix = transpose(X_gen_matrix)
size(X_gen_matrix)

(4, 1049)

In [165]:
X_cur_matrix = value.(X_cur);
    X_cur_matrix = transpose(X_cur_matrix)
size(X_cur_matrix)

(4, 1049)

In [166]:
using Plots

@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1, time_ls]
b = X_gen_matrix[2, time_ls]
c = X_gen_matrix[3, time_ls]
d = X_gen_matrix[4, time_ls]
a_c = X_cur_matrix[1, time_ls] * -1
b_c = X_cur_matrix[2, time_ls] * -1
c_c = X_cur_matrix[3, time_ls] * -1
d_c = X_cur_matrix[4, time_ls] * -1
x = time_ls

plotly()
stackedarea(x, [d c b a], labels=["Coal" "CCGT" "Wind" "Solar"], lw=0)
stackedarea!(x, [d_c c_c b_c a_c], labels=["Coal, curtail" "CCGT, curtial" "Wind, curtail" "Solar, curtail"], lw=0)
plot!(x, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
# plot!(x, value.(X_D)[:], label="Demand, shifted", lc=:black, lw=2)
plot!(size=(800,400))

In [167]:
# Compare availability of solar to wind
plotly()
plot(x, total_demand[time_ls], label="Demand", lc=:black, lw=1)
plot!(x, value.(A_s)[time_ls, 1]*P_max_w, label="Solar", lc=:orange, lw=1)
plot!(x, value.(A_s)[time_ls, 2]*P_max_w, label="Wind", lc=:blue, lw=1)
plot!(size=(1000,400))